## Tier 1. Module 3: Basic Algorithms and Data Structures

## Topic 5 - Search algorithms

## Homework

### Task 1


Add a `delete` method to delete key-value pairs of the `HashTable` table, which is implemented in the outline.

In [63]:
class HashTable:
    def __init__(self, size: int):
        self.size = size
        self.table = [[] for _ in range(self.size)]

    def hash_function(self, key):
        return hash(key) % self.size

    def insert(self, key, value):
        key_hash = self.hash_function(key)
        key_value = [key, value]

        if self.table[key_hash] is None:
            self.table[key_hash] = [key_value]
            return True
        else:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    pair[1] = value
                    return True
            self.table[key_hash].append(key_value)
            return True

    def get(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for pair in self.table[key_hash]:
                if pair[0] == key:
                    return pair[1]
        return None

    def delete(self, key):
        key_hash = self.hash_function(key)
        if self.table[key_hash] is not None:
            for index, pair in enumerate(self.table[key_hash]):
                if pair[0] == key:
                    del self.table[key_hash][index]
                    return True

    def all(self):
        print(self.table)


# Testing
H = HashTable(5)
H.insert("apple", 10)
H.insert("orange", 20)
H.insert("banana", 30)

H.all()  # [[], [['orange', 20]], [], [], [['apple', 10], ['banana', 30]]]

H.delete("orange")
print(H.get("orange"))  # return: None

H.delete("banana")
print(H.get("banana"))  # return: None

H.delete("blabla")
print(H.get("blabla"))  # return: None

H.all()  # [[], [], [], [], [['apple', 10]]]

[[['apple', 10], ['orange', 20], ['banana', 30]], [], [], [], []]
None
None
None
[[['apple', 10]], [], [], [], []]


### Task 2


Implement a binary search for a sorted array with fractional numbers. A function written for binary search should return a tuple where the first element is the number of iterations required to find the element. The second element should be the "upper limit" - this is the smallest element that is greater than or equal to the given value.

In [64]:
def binary_search(array: list, target: float) -> tuple[int, float]:
    left_index = 0
    right_index = len(array) - 1
    iterations = 0

    while left_index <= right_index:
        mid_index = left_index + (right_index - left_index) // 2
        iterations += 1

        if target == array[mid_index]:
            return iterations, array[mid_index]

        elif target > array[mid_index]:
            left_index = mid_index + 1

        else:
            right_index = mid_index - 1

    # If the exact target is not present in the array
    if left_index < len(array):  # The nearest larger value
        upper_limit = array[left_index]
    else:  # If target is larger than max element of the array
        upper_limit = None

    return iterations, upper_limit


# Testing
arr = [0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5]
target = 1.4
iterations, upper_limit = binary_search(arr, target)
print("Iterations:", iterations)
print("Upper limit:", upper_limit)

Iterations: 4
Upper limit: 1.5


### Task 3


Compare the performance of Boyer-Moore, Knuth-Morris-Pratt and Rabin-Karp substring search algorithms based on two text files ([article 1](https://drive.google.com/file/d/18_R5vEQ3eDuy2VdV3K5Lu-R-B-adxXZh/view), [article 2](https://drive.google.com/file/d/13hSt4JkJc11nckZZz2yoFHYL89a4XkMZ/view)). Using timeit, it is necessary to measure the execution time of each algorithm for two types of substrings: one that actually exists in the text, and the other - a fictional one (the choice of substrings is your choice). Based on the received data, determine the fastest algorithm for each text separately and as a whole.

#### 3.1 - Boyer-Moore algorithm


In [65]:
def build_shift_table(pattern: str) -> dict:
    """
    Create a shift table for the Boyer-Moore algorithm
    """
    table = {}
    length = len(pattern)
    # For each character in the substring, set an offset equal to the length of the substring
    for index, char in enumerate(pattern[:-1]):
        table[char] = length - index - 1
    # If the character is not in the table, the offset will be equal to the length of the substring
    table.setdefault(pattern[-1], length)
    return table


def boyer_moore_search(main_string: str, pattern: str) -> int:
    # Create a table of shifts for the pattern (substring)
    shift_table = build_shift_table(pattern)
    i = 0  # Initialize the initial index for the main text

    # Go through the main text, comparing with the substring
    while i <= len(main_string) - len(pattern):
        j = len(pattern) - 1  # Start from the end of the substring

        # Compare characters from the end of the substring to its beginning
        while j >= 0 and main_string[i + j] == pattern[j]:
            j -= 1  # Move to the beginning of the substring

        # If the entire substring matches, return its position in the text
        if j < 0:
            return i

        # Shift the index i based on the shift table
        # This allows to "jump" over non-matching parts of the text
        i += shift_table.get(main_string[i + len(pattern) - 1], len(pattern))

    # If the substring is not found, return -1
    return -1

#### 3.2 - Knuth-Morris-Pratt algorithm


In [66]:
def compute_lps(pattern: str) -> list:
    """
    Computes lps list (longest prefix suffixes)
    """
    lps = [0] * len(pattern)
    length = 0
    i = 1

    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
        else:
            if length != 0:
                length = lps[length - 1]
            else:
                lps[i] = 0
                i += 1

    return lps


def kmp_search(main_string: str, pattern: str) -> int:
    M = len(pattern)
    N = len(main_string)

    lps = compute_lps(pattern)

    i = j = 0

    while i < N:
        if pattern[j] == main_string[i]:
            i += 1
            j += 1
        elif j != 0:
            j = lps[j - 1]
        else:
            i += 1

        if j == M:
            return i - j

    return -1  # If the substring is not found

#### 3.3 - Rabin-Karp algorithm


In [67]:
def polynomial_hash(string: str, base: int = 256, modulus: int = 101) -> int:
    """
    Returns the polynomial hash of string
    """
    n = len(string)
    hash_value = 0
    for i, char in enumerate(string):
        power_of_base = pow(base, n - i - 1) % modulus
        hash_value = (hash_value + ord(char) * power_of_base) % modulus
    return hash_value


def rabin_karp_search(main_string: str, pattern: str) -> int:
    # Main string and search substring lengths
    substring_length = len(pattern)
    main_string_length = len(main_string)

    # Hash base number and modulus
    base = 256
    modulus = 101

    # The hash value for the search substring and the current segment in the main string
    substring_hash = polynomial_hash(pattern, base, modulus)
    current_slice_hash = polynomial_hash(main_string[:substring_length], base, modulus)

    # Default value for hash recalculation
    h_multiplier = pow(base, substring_length - 1) % modulus

    # Run through the main line
    for i in range(main_string_length - substring_length + 1):
        if substring_hash == current_slice_hash:
            if main_string[i : i + substring_length] == pattern:
                return i

        if i < main_string_length - substring_length:
            current_slice_hash = (
                current_slice_hash - ord(main_string[i]) * h_multiplier
            ) % modulus
            current_slice_hash = (
                current_slice_hash * base + ord(main_string[i + substring_length])
            ) % modulus
            if current_slice_hash < 0:
                current_slice_hash += modulus

    return -1

#### 3.4 - Input data


In [68]:
import chardet
import requests

url_1 = (
    "https://drive.google.com/file/d/18_R5vEQ3eDuy2VdV3K5Lu-R-B-adxXZh/view?usp=sharing"
)
url_2 = (
    "https://drive.google.com/file/d/13hSt4JkJc11nckZZz2yoFHYL89a4XkMZ/view?usp=sharing"
)


def get_txt_file(url: str) -> str:
    """
    The function of extracting text from files on Google Drive

    :param url: link to the file
    :return: extracted and decoded text
    """
    file_id = url.split("/")[-2]
    request_url = "https://drive.google.com/uc?id=" + file_id
    response = requests.get(request_url)
    raw_data = response.content
    encoding = chardet.detect(raw_data)["encoding"]
    return raw_data.decode(encoding)


file_1 = get_txt_file(url_1)
file_2 = get_txt_file(url_2)

#### 3.5 - Patterns preparation


In [77]:
import random
import re
import string


def prepare_patterns(text: str) -> list:
    """
    A function that randomly selects 5 strings from the text and
    creates 5 random strings that are not in the text

    :param text: the text from which 5 random strings are extracted,
    each string contains 3 words
    :retun: a list with 5 existing and 5 non-existent strings in
    the text
    """
    results = []

    # Preparation of existing patterns
    substrings = re.findall(r"\b\w+\b \b\w+\b \b\w+\b", text)
    samples = random.sample(substrings, 5)
    for sample in samples:
        results.append(sample.lower())

    # Preparation of non-existent patterns
    characters = string.ascii_letters
    for i in range(5):
        random_string = "".join(random.choice(characters) for _ in range(10))
        results.append(random_string.lower())

    return results


test_patterns_1 = prepare_patterns(file_1)
test_patterns_2 = prepare_patterns(file_2)

print(test_patterns_1)
print(test_patterns_2)

['бібліотеках мов програмування', 'експоненціальний пошук використовується', 'потрібно обрати чергові', 'щоб їх було', 'досягнення локального мінімуму', 'ykurcinrjl', 'apdjcbvtfe', 'rneazqiawq', 'dvwzxzgkah', 'zrxlgxjbkd']
['на цьому етапі', 'послідовному розташуванню елементів', 'зору якості її', 'є можливість відфільтрувати', 'здійснюється пошук усіх', 'tabymbtgub', 'dramqhwexd', 'kxoywydbrh', 'oouxjsjamo', 'cysvqjbnrf']


#### 3.6 - Testing algorithms


In [79]:
import timeit


def test_algorithms(algorithms: dict, main_string: str, patterns: list) -> dict:
    """
    Function for testing searching algorithms on a text

    :param algorithms: a dictionary, where the keys are algorithm names, and
    the values are functions or methods with an implemented searching algorithm
    :param main_string: text where the search for patterns will be carried out
    :param paterns: a list of strings to search for in the text
    :return: a dictionary with algorithm names as keys and times spent for
    searching as values
    """
    results = {}
    for algo_name, algo_func in algorithms.items():
        results[algo_name] = 0
        for pattern in patterns:
            # time_taken = timeit.timeit(lambda: algo_func(main_string, pattern), number=10)
            time_taken = measure_search_time(algo_func, main_string, pattern)
            results[algo_name] += time_taken
    return results


def measure_search_time(func, main_string: str, pattern: str) -> int:
    """
    The function of calculating the time required to search with a specific
    functionfor a pattern in the text

    :param function: a function with an implemented searching algorithm
    :param main_string: text where the search for patterns will be carried out
    :param patern: a strings to search for in the text
    :return: time of search
    """
    setup_code = f"""from __main__ import {func.__name__}"""
    stmt = f"{func.__name__}(main_string, pattern)"
    return timeit.timeit(
        stmt,
        setup=setup_code,
        globals={"main_string": main_string, "pattern": pattern},
        number=10,
    )

In [80]:
algorithms = {
    "Boyer-Moore": boyer_moore_search,
    "Knuth-Morris-Pratt": kmp_search,
    "Rabin-Karp": rabin_karp_search,
}

Testing on the first file


In [81]:
results_1 = test_algorithms(algorithms, file_1.lower(), test_patterns_1)
for algo, timings in results_1.items():
    print(f"Algorithm: {algo:<20} Time taken: {timings:.6f} seconds")

Algorithm: Boyer-Moore          Time taken: 0.032971 seconds
Algorithm: Knuth-Morris-Pratt   Time taken: 0.117601 seconds
Algorithm: Rabin-Karp           Time taken: 0.288353 seconds


Testing on the second file


In [82]:
results_2 = test_algorithms(algorithms, file_2.lower(), test_patterns_2)
for algo, timings in results_2.items():
    print(f"Algorithm: {algo:<20} Time taken: {timings:.6f} seconds")

Algorithm: Boyer-Moore          Time taken: 0.046305 seconds
Algorithm: Knuth-Morris-Pratt   Time taken: 0.146952 seconds
Algorithm: Rabin-Karp           Time taken: 0.408292 seconds


#### Conclusion:

As can be seen from the testing of both text files, the Boyer-Moore algorithm is significantly faster than all the others. This can be explained by the fact that it has a more efficient shifting strategy by searching the substring from right to left and a special shifting table that shows how much to shift the substring if the characters in the text search do not match. In some circumstances, when no substring characters match the text characters, forcing the algorithm to make the maximum possible shift on each failed match attempt, the Boyer-Moore algorithm can even achieve linear complexity `O(n)`, where `n` is the length of the text.